In [6]:
import BandaiDataset as bd

In [7]:
MODEL_DIR = "./saved_models"
DATASET_DIR = "./datasets/data/"
FILELIST_PATH = "datafiles.txt"

In [8]:
dataset = bd.BandaiDataset(FILELIST_PATH)

In [9]:
import torch
set_frame = 50
def custom_collate_fn(batch):
    motion_batch_tensor = torch.FloatTensor(len(batch),50,480,640)
    motion_tensors = []
    labels = []
    #print(type(batch))

    for item in batch:
        #print(item)
        motion_tensor = item.get_motion_tensor(50) # load an motion as a tensor(frames,width,height)
        motion_tensors.append(motion_tensor.unsqueeze(0)) # put motions into a list : to be checked 
        labels.append(item.label)

    torch.cat(motion_tensors, out=motion_batch_tensor)
    label_batch_tensor = torch.LongTensor(labels)
    return (motion_batch_tensor,label_batch_tensor)

In [10]:
from torch.utils.data import DataLoader, random_split
import multiprocessing as mp

def load_data(file_list_path= '', data_path='', batch_sz = 5, train_val_test_split = [0.7,0.1,0.2]):
    assert sum(train_val_test_split) == 1, "Train, val and test fractions should sum to 1!" 
    dataset = bd.BandaiDataset(data_path)
    dataset.load()

    tr_va_te = []
    n_cpus = mp.cpu_count()
    
    for frac in train_val_test_split:
        num = round(frac * dataset.num_of_files)
        tr_va_te.append(num)
    
    if tr_va_te[0] != (dataset.num_of_files - tr_va_te[1] - tr_va_te[2]):
        tr_va_te[0] = (dataset.num_of_files - tr_va_te[1] - tr_va_te[2])
    
    #assert 1==2, f"tr_va_te = {tr_va_te}"

    train_split, val_split, test_split = random_split(dataset, tr_va_te)

    train_dl = DataLoader(train_split,
                          batch_size=batch_sz,
                          shuffle=True,
                          collate_fn=custom_collate_fn,
                          num_workers=n_cpus
                        )
    val_dl = DataLoader(val_split,
                        batch_size=batch_sz,
                        shuffle=True,
                        collate_fn=custom_collate_fn,
                        num_workers=n_cpus)
    test_dl = DataLoader(test_split,
                         batch_size=batch_sz,
                         shuffle=True,
                         collate_fn=custom_collate_fn,
                         num_workers=n_cpus)

    return train_dl, val_dl, test_dl

In [11]:
train_dl, val_dl, test_dl = load_data()

dataset-1_bow_active_001: stream end
129
dataset-1_bow_angry_001: stream end
91
dataset-1_bow_childish_001: stream end
99
dataset-1_bow_chimpira_001: stream end
118
dataset-1_bow_feminine_001: stream end
219
dataset-1_bow_giant_001: stream end
123
dataset-1_bow_happy_001: stream end
139
dataset-1_bow_masculinity_001: stream end
71
dataset-1_bow_musical_001: stream end
157
dataset-1_bow_normal_001: stream end
139
dataset-1_bow_not-confident_001: stream end
146
dataset-1_bow_old_001: stream end
173
dataset-1_bow_proud_001: stream end
138
dataset-1_bow_sad_001: stream end
222
dataset-1_bow_tired_001: stream end
166
dataset-1_byebye_active_001: stream end
230
dataset-1_byebye_angry_001: stream end
56
dataset-1_byebye_childish_001: stream end
170
dataset-1_byebye_chimpira_001: stream end
150
dataset-1_byebye_feminine_001: stream end
85
dataset-1_byebye_giant_001: stream end
236
dataset-1_byebye_happy_001: stream end
221
dataset-1_byebye_masculinity_001: stream end
202
dataset-1_byebye_music

In [12]:
train_motions, _ = next(iter(train_dl))

In [13]:
test_motions,test_labels = next(iter(test_dl))

In [14]:
'''
for (motion_batch,label_batch) in train_dl:
    batch_sz = len(motion_batch)
    print(batch_sz)
'''

'\nfor (motion_batch,label_batch) in train_dl:\n    batch_sz = len(motion_batch)\n    print(batch_sz)\n'

In [18]:
import numpy as np
print(np.size(dataset[164].pose_list[0]))
print(dataset[164].frame_num)
#tensor = np.array(dataset[54].pose_list)
for pose in dataset[54].pose_list:
    print(np.size(pose))

307200
227
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200


In [19]:
print(len(dataset.filelist))
print(dataset.filelist[164])
dataset.num_of_files


165
dataset-1_walk_tired_002


165

In [20]:
for i in range(0,165):
    print(f'{i}: {dataset[i].frame_num} ->' ,end='')
    tensor = dataset[i].get_motion_tensor(50)
    print({dataset[i].frame_num})

0: 129 ->{50}
1: 91 ->{50}
2: 99 ->{50}
3: 118 ->{50}
4: 219 ->{50}
5: 123 ->{50}
6: 139 ->{50}
7: 71 ->{50}
8: 157 ->{50}
9: 139 ->{50}
10: 146 ->{50}
11: 173 ->{50}
12: 138 ->{50}
13: 222 ->{50}
14: 166 ->{50}
15: 230 ->{50}
16: 56 ->{50}
17: 170 ->{50}
18: 150 ->{50}
19: 85 ->{50}
20: 236 ->{50}
21: 221 ->{50}
22: 202 ->{50}
23: 158 ->{50}
24: 122 ->{50}
25: 141 ->{50}
26: 123 ->{50}
27: 80 ->{50}
28: 235 ->{50}
29: 304 ->{50}
30: 186 ->{50}
31: 111 ->{50}
32: 124 ->{50}
33: 131 ->{50}
34: 329 ->{50}
35: 143 ->{50}
36: 224 ->{50}
37: 213 ->{50}
38: 188 ->{50}
39: 112 ->{50}
40: 209 ->{50}
41: 142 ->{50}
42: 174 ->{50}
43: 254 ->{50}
44: 211 ->{50}
45: 86 ->{50}
46: 75 ->{50}
47: 76 ->{50}
48: 70 ->{50}
49: 98 ->{50}
50: 68 ->{50}
51: 118 ->{50}
52: 71 ->{50}
53: 68 ->{50}
54: 21 ->{50}
55: 100 ->{50}
56: 86 ->{50}
57: 106 ->{50}
58: 82 ->{50}
59: 94 ->{50}
60: 141 ->{50}
61: 111 ->{50}
62: 120 ->{50}
63: 187 ->{50}
64: 301 ->{50}
65: 164 ->{50}
66: 106 ->{50}
67: 153 ->{50}
68: 201 

In [21]:
import torchmetrics.classification as tmcls 
class ClassifierMetrics(object):
    ap: float
    precision: float
    recall: float
    f1: float
    acc: float
    count: int

    def __init__(self, task, n_labels, device):
        self.task = task
        if self.task == "multiclass":
            self.ap_metric = tmcls.MulticlassAveragePrecision(num_classes=n_labels, average=None, thresholds=None).to(device)
            self.precision_metric = tmcls.MulticlassPrecision(num_classes=n_labels).to(device)
            self.recall_metric = tmcls.MulticlassRecall(num_classes=n_labels).to(device)
            self.f1_metric = tmcls.MulticlassF1Score(num_classes=n_labels).to(device)
            self.acc_metric = tmcls.MulticlassAccuracy(num_classes=n_labels).to(device)

        elif self.task == "multilabel":
            self.ap_metric = tmcls.MultilabelAveragePrecision(num_labels=n_labels, average=None, thresholds=None).to(device)
            self.precision_metric = tmcls.MultilabelPrecision(num_labels=n_labels).to(device)
            self.recall_metric = tmcls.MultilabelRecall(num_labels=n_labels).to(device)
            self.f1_metric = tmcls.MultilabelF1Score(num_labels=n_labels).to(device)
            self.acc_metric = tmcls.MultilabelAccuracy(task=self.task, num_labels=n_labels).to(device)
        self.reset()
    

    def reset(self):
        self.ap = 0
        self.precision = 0
        self.recall = 0
        self.f1 = 0
        self.acc = 0
        self.count = 0

    def update(self, y_pred, y):
        y = y.long()
        self.ap += self.ap_metric(y_pred, y)
        self.precision += self.precision_metric(y_pred, y)
        self.recall += self.recall_metric(y_pred, y)
        self.f1 += self.f1_metric(y_pred, y)
        self.acc += self.acc_metric(y_pred, y)
        self.count += 1 #y.size(0)

    def calc(self, y_pred, y):
        self.reset()
        y = y.long()
        self.ap = self.ap_metric(y_pred, y)
        self.precision = self.precision_metric(y_pred, y)
        self.recall = self.recall_metric(y_pred, y)
        self.f1 = self.f1_metric(y_pred, y)

    def avg(self):
        self.ap = self.ap / self.count
        self.precision = self.precision / self.count
        self.recall = self.recall / self.count
        self.f1 = self.f1 / self.count
        self.acc = self.acc / self.count

In [44]:
import torch.nn as nn
from torchsummary import summary
def get_simple_linear_net():
    return nn.Sequential(
        nn.Flatten(),                # Input is a 2d array of pixel values, so we flatten it out to 1d
        nn.Linear(50*640*480, 128),       # Input layer connects each input node to each hidden node. MNIST images are 28*28 pixels, hidden size can be anything we want
        nn.ReLU(),                   # ReLU activation only lets a signal through if it is > 0
        nn.Linear(128, 10)  # Output connects each node in the hidden layer to 10 output classes - the number of digits we want to classify!
        
    )

summary(get_simple_linear_net(), input_size=(50,640, 480), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1             [-1, 15360000]               0
            Linear-2                  [-1, 128]   1,966,080,128
              ReLU-3                  [-1, 128]               0
            Linear-4                   [-1, 10]           1,290
Total params: 1,966,081,418
Trainable params: 1,966,081,418
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 58.59
Forward/backward pass size (MB): 117.19
Params size (MB): 7500.01
Estimated Total Size (MB): 7675.79
----------------------------------------------------------------


In [45]:
def train_model(model,epochs,train_dl,optimiser):
    msg = ""
    for epoch in range (epochs):
        total_steps = len(train_dl)
        correct = 0
        total = 0
        model.train()
        for batch_num, (motion_batch, label_batch) in enumerate(train_dl):
            batch_sz = len(motion_batch)
            output
